#### Useful imports and extensions

In [1]:
import jsonlines
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pyspark import SparkContext
import numpy as np
import json
from tqdm import tqdm
import pyarrow.feather as feather

%load_ext autoreload

## Loading dataset and extracting climate change related videos

We want to study youtube videos related to climate change : the first step is to filter our broad dataset of all videos.
For this, we generated a list of keywords related to climate change :

In [ ]:
keywords = [
    'climate change', 'global warming', 'greenhouse gases', 'rising sea levels',
    'extreme weather', 'temperature increase', 'climate action', 
    'carbon emissions', 'renewable energy', 'sustainable living',
    'climate science', 'climate crisis', 'climate adaptation',
    'natural disaster', 'extreme heat', 'melting ice cap', 'sea level rise',
    'biodiversity loss', 'deforestation', 'ocean acidification', 'climate policy', 
    'environmental policy',
    'disaster recovery', 'climate refugees',
    'carbon footprint', 'sustainable development', 'green technology',
    'renewable resource', 'eco-friendly',
    'ecosystem disruption', 'impact on climate',
    'paris agreement', 'climate awareness',
    'environmental justice', 'clean energy', 'zero carbon',
    'green infrastructure', 'ozone layer',
    'pollution', 'water scarcity', 'climate education'
]

We create functions to check if a video contains climate change related keywords in its title, description or tags

In [ ]:
def about_climate(tags):
    tags_set = set(tags)
    return any(tag.strip().lower() in keywords for tag in tags_set)

def climate_text(text):
    lowercase_text = text.lower()
    return any(keyword in lowercase_text for keyword in keywords)

def climate_related(video):
    return (climate_text(video['title']) or about_climate(video['tags']) or climate_text(video['description']))

Since the original video metadata dataset is huge (97 GB), we read and filter the dataset using pyskark in order to get a reduced dataset with only the relevant videos.

#### Load the dataset

In [ ]:
path_to_disk = '/Volumes/Maxtor'
file_path = '/yt_metadata_en.jsonl'

sc = SparkContext()
raw_data = sc.textFile(path_to_disk + file_path)
video_dataset = raw_data.map(lambda x: json.loads(x))

#### Filter the dataset

In [ ]:
climate_videos = video_dataset.filter(climate_related)
climate_videos_collected = climate_videos.collect()

In [ ]:
climate_videos_path = "data/coarse_filter_climate_videos.jsonl"
with jsonlines.open(climate_videos_path, "w") as jsonl_file:
   jsonl_file.write_all(climate_videos_collected)

#### Sample dataset

We create 2 random sample dataset (1% ands 0.1%) which may be useful later in our analysis for statisitical comparison with the cliamte related videos dataset.

In [ ]:
seed = 42

random_sample_1percent = video_dataset.sample(False, 0.01, seed=seed)
random_sample_1percent_collected = random_sample_1percent.collect()
sample_1percent_path = "data/sample_1percent.jsonl"
with jsonlines.open(sample_1percent_path, "w") as jsonl_file:
    jsonl_file.write_all(random_sample_1percent_collected)


random_sample_01percent = video_dataset.sample(False, 0.001, seed=seed)
random_sample_01percent_collected = random_sample_01percent.collect()
sample_01percent_path = "data/sample_01percent.jsonl"
with jsonlines.open(sample_01percent_path, "w") as jsonl_file:
    jsonl_file.write_all(random_sample_01percent_collected)


In [ ]:
sc.stop()

### Extracting useful features from the whole dataset

We keep working on the whole dataset in order to extract useful features which may be useful later for comparison purpose. We use the .feather helper for this. 

In [2]:
feather_path = "data/yt_metadata_helper.feather"
video_dataset_feather = pd.read_feather(feather_path)

In [8]:
video_count_by_date_cat = video_dataset_feather.groupby(['upload_date', 'categories']).size().reset_index(name='count')
video_count_by_date = video_count_by_date_cat[['upload_date', 'count']].groupby(['upload_date']).sum().reset_index()
video_count_by_cat = video_count_by_date_cat[['categories', 'count']].groupby(['categories']).sum().reset_index()


In [9]:
video_count_by_date_cat.to_json("data/nb_videos_by_date_cat.jsonl", orient="records", lines=True)
video_count_by_date.to_json("data/nb_videos_by_date.jsonl", orient="records", lines=True)
video_count_by_cat.to_json("data/nb_videos_by_cat.jsonl", orient="records", lines=True)